In [ ]:
import pandas as pd
import datapackage

p = datapackage.Package('../datapackage.json')
dfs = {r.name: pd.DataFrame(r.read(keyed=True)) for r in p.resources}

In [ ]:
prod1 = dfs['production_ecsc'].set_index(['year', 'product']).loc[2016, 'mass']
prod2 = dfs['production_derived'].set_index(['year', 'product']).loc[2016, 'mass']
prod2_in = dfs['production_ecsc_for_derived'].set_index(['year', 'product']).loc[2016, 'mass']
exports = dfs['exports'].set_index(['year', 'product']).loc[2016, 'mass']

In [ ]:
prod1.add(prod2, fill_value=0).sub(prod2_in, fill_value=0).sub(exports, fill_value=0)

In [ ]:
supply = dfs['supply'].groupby(['product']).sum()
supply

In [ ]:
flows = []
for i, x in prod1.iteritems():
    e = exports[i]
    try:
        f = prod2_in[i]  # feedstocks for derived products
    except KeyError:
        f = 0
    flows.append(('uk_production', i, i, x))
    flows.append((i, 'export', i, e))
    flows.append((i, 'deriv', i, f))
    flows.append((i, 'uk_market', i, x - e - f))
    
for i, x in prod2.iteritems():
    e = exports[i]
    flows.append(('deriv', i, i, x))
    flows.append((i, 'export', i, e))
    flows.append((i, 'uk_market', i, x - e))
        
flows = pd.DataFrame(flows, columns=('source', 'target', 'material', 'value'))
flows

In [ ]:
from floweaver import *

pm = Partition.Simple('process', 
['Ingots, blooms, billets and slabs',
 'Rounds and squares for tubes',
 'Rods and bars for reinforcement',
 'Rods in coil',
 'Hot rolled bars in lengths',
 'Heavy sections, sheet piling, rails and rolled accessories',
 'Light sections',
 'Plates in coil and in lengths',
 'Sheets, coated and uncoated',
 'Bright bars', 'Cold rolled narrow strip', 'Tubes and pipes'])

pmm = Partition.Simple('material', 
['Ingots, blooms, billets and slabs',
 'Rounds and squares for tubes',
 'Rods and bars for reinforcement',
 'Rods in coil',
 'Hot rolled bars in lengths',
 'Heavy sections, sheet piling, rails and rolled accessories',
 'Light sections',
 'Plates in coil and in lengths',
 'Sheets, coated and uncoated',
 'Bright bars', 'Cold rolled narrow strip', 'Tubes and pipes'])

nodes = {
    'prod1': ProcessGroup(['uk_production']),
    'semi1': ProcessGroup(['Ingots, blooms, billets and slabs',
 'Rounds and squares for tubes',
 'Rods and bars for reinforcement',
 'Rods in coil',
 'Hot rolled bars in lengths',
 'Heavy sections, sheet piling, rails and rolled accessories',
 'Light sections',
 'Plates in coil and in lengths',
 'Sheets, coated and uncoated'], pm),
    'prod2': ProcessGroup(['deriv']),
    'semi2': ProcessGroup(['Bright bars', 'Cold rolled narrow strip', 'Tubes and pipes'], pm),
    'export': ProcessGroup(['export']),
    'uk_market': ProcessGroup(['uk_market']),
    
    'e1': Waypoint(),
}

ordering = [
    [[], ['prod1']],
    [[], ['semi1']],
    [['e1'], ['prod2']],
    [[], ['semi2']],
    [['export'], ['uk_market']],
]

bundles = [
    Bundle('prod1', 'semi1'),
    Bundle('semi1', 'prod2'),
    Bundle('prod2', 'semi2'),
    Bundle('semi1', 'export', waypoints=['e1']),
    Bundle('semi2', 'export'),
    Bundle('semi1', 'uk_market'),
    Bundle('semi2', 'uk_market'),
]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=pmm)
weave(sdd, flows).to_widget(width=1000, height=600) \
.auto_save_svg('sankey.svg')